In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import pytorch_lightning as pl
import random

class MyLightningModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.pad = nn.ZeroPad2d(3)  # padding de 2 píxeles a cada lado
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 4, kernel_size = 4)  # imagen de 1 canal, output 16 filtros, kernel 3x3
        self.conv2 =
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 nn.Conv2d(3, 12,)

        self.fc1 = nn.Linear(16 * 14 * 14, 10)  # ajusta esto según el tamaño tras convoluciones

    def forward(self, x):
        # Zero Padding
        x = self.pad(x)

        # Rotación aleatoria entre -30° y 30°
        angle = random.uniform(-30, 30)
        x = TF.rotate(x, angle)

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torchvision.transforms as T

class CNNModel(pl.LightningModule):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Transformaciones
        self.augment = T.RandomRotation(degrees=20)

        # Zero Padding: de 21x21 → 27x27
        self.zeropad = nn.ConstantPad2d(3, 0)

        # Convoluciones iniciales
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)

        # Max pooling
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convoluciones intermedias
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3)

        # Segundo max pooling
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected
        self.fc1 = nn.Linear(2304, 64)
        self.fc2 = nn.Linear(64, 5)  # 5 clases de salida

    def forward(self, x):
        x = self.zeropad(x)                # [batch, 3, 27, 27]
        x = self.augment(x)                # Rotación

        x = F.relu(self.conv1(x))          # [batch, 32, 24, 24]
        x = F.relu(self.conv2(x))          # [batch, 32, 24, 24]
        x = self.pool1(x)                  # [batch, 32, 12, 12]

        x = F.relu(self.conv3(x))          # [batch, 64, 10, 10]
        x = F.relu(self.conv4(x))          # [batch, 64, 8, 8]
        x = self.pool2(x)                  # [batch, 64, 6, 6]

        x = torch.flatten(x, start_dim=1)  # [batch, 2304]
        x = F.relu(self.fc1(x))            # [batch, 64]
        x = self.fc2(x)                    # [batch, 5]
        return F.softmax(x, dim=1)         # Softmax → Probabilidades por clase


In [28]:
import torch
from torch.utils.data import Dataset
import pandas as pd

import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((21, 21)),  # fuerza tamaño
    transforms.ToTensor()
])



pkl_path = 'dataset_ZTF.pkl'
class ZTFDataset(Dataset):
    def __init__(self, pkl_path, transform=None):
        self.df = pd.read_pickle(pkl_path)
        self.transform = transform
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = torch.tensor([
            row['science'],
            row['reference'],
            row['diff']
        ])  # [3, 21, 21]
        label = row['class']
        label_map = {'bogus':0, 'agn':1, 'asteroid':2, 'vs':3, 'sn':4, 'satellite':5}
        return image.float(), label_map[label]



In [29]:
model = ZTFClassifier()


In [30]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F

class ZTFClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.pad = nn.ZeroPad2d(3)  # de 21x21 a 27x27
        #self.rotation = nn.RandomRotation(degrees=10)

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4),  # 24x24x32
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3),  # 22x22x32
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 11x11x32

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # 11x11x64
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # 11x11x64
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # 11x11x64
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2)  # 5x5x64
        )

        self.fc = nn.Sequential(
            nn.Flatten(),  # 5x5x64 = 1600
            nn.Linear(1600, 64),
            nn.ReLU(),
            nn.Linear(64, 6),  # 6 clases
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.pad(x)
        #x = self.rotation(x)
        x = self.conv_layers(x)
        x = self.fc(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


In [31]:
from torch.utils.data import random_split, DataLoader



# Carga el dataset completo
dataset = ZTFDataset("dataset_ZTF.pkl", transform=transform)


# Divide en entrenamiento y validación (80%-20%)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)


MemoryError: 

In [13]:
model = ZTFClassifier()


In [14]:
from pytorch_lightning import Trainer

trainer = Trainer(
    max_epochs=10,           # Número de épocas
    accelerator="cpu",       # O "cpu" si no tienes GPU
    devices=1,               # Número de GPUs (si aplica)
    log_every_n_steps=10     # Frecuencia de logs
)


You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [8]:
trainer.fit(model, train_loader, val_loader)


c:\Users\Arkaz\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.

  | Name        | Type       | Params | Mode 
---------------------------------------------------
0 | pad         | ZeroPad2d  | 0      | train
1 | conv_layers | Sequential | 103 K  | train
2 | fc          | Sequential | 102 K  | train
---------------------------------------------------
205 K     Trainable params
0         Non-trainable params
205 K     Total params
0.824     Total estimated model params size (MB)
20        Modules in train mode
0         Modules in eval mode
c:\Users\Arkaz\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoa

Training: |          | 0/? [00:00<?, ?it/s]

C:\Users\Arkaz\AppData\Local\Temp\ipykernel_816\916995192.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  image = torch.tensor([


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x16384 and 1600x64)

In [15]:
img = next(iter(train_loader))[0]  # batch de imágenes
print(img.shape)


RuntimeError: stack expects each tensor to be equal size, but got [3, 63, 63] at entry 0 and [3, 59, 63] at entry 59